In [20]:
""" IGNORE CELL - Dev Environment setup (Not necessary if you've installed via pip) """
from os.path import join, dirname, abspath
from os import getenv, getcwd
import sys

module_path = abspath(join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from dotenv import load_dotenv
from alpaca_finance.automated_vault import AutomatedVaultPosition

In [3]:
# Variables:
load_dotenv()

PUBKEY = getenv('PUBLIC_WALLET_ADDRESS', "<or_hardcode_here>")
PRIVKEY = getenv('PRIVATE_WALLET_KEY', "<or_hardcode_here>")
assert PUBKEY is not None and PRIVKEY is not None, "Public and private wallet key required to sign withdraw txn"

POSITION_KEY = "L3x-BUSDBTCB-PCS2"  # <-- Replace with your own Automated Vault position key (BTCB Savings (3x) in this case)

In [4]:
# Create a automated vault position instance:
position = AutomatedVaultPosition(position_key=POSITION_KEY, owner_wallet_address=PUBKEY, owner_wallet_key=PRIVKEY)

position.name, position.key, position.shares()[1]

('Long 3x BUSD-BTCB PCS2', 'l3x-busdbtcb-pcs2', 10.599732233561554)

### Withdraw 2 Shares from the Position (Using the 'Minimize Trading' strategy):

In [6]:
# Must supply the shares in integer value:
shares = position.to_wei(2, position.bep20_vault_token.decimals())
shares

2000000000000000000

In [8]:
# We don't need to specify a strategy here, since the default strategy is "Minimize Trading":
txn_receipt = position.do_withdraw(shares)
txn_receipt

{'value': 0, 'gas': 4889680, 'from': '0xC9E6e248928aC18eD6b103653cBcF66d23B743C6', 'chainId': 56, 'gasPrice': 5000000000, 'nonce': 17, 'to': '0xA1679223b7585725aFb425a6F59737a05e085C40', 'data': '0xfe55892d0000000000000000000000000000000000000000000000001bc16d674ec80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000080000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000190000000000000000000000000000000000000000000000000000000000000001'}


TransactionReceipt(transactionHash=HexBytes('0xc23e2ed4fe003a64b249ce6ea9e38108df7715d9011e2629bbb59513056623f5'), blockHash=HexBytes('0x68e1c1eb00fe978026abf1d84995ef7b94906cbee31680630f4d4fa5c0d813d8'), blockNumber=19784660, contractAddress=None, cumulativeGasUsed=12123140, gasSpendUSD=6.434590946335803, fromAddress='0xC9E6e248928aC18eD6b103653cBcF66d23B743C6', toAddress='0xA1679223b7585725aFb425a6F59737a05e085C40', status=1, transactionIndex=51, type='0x0', effectiveGasPrice=None)

In [11]:
f"Gas Spend USD: ${txn_receipt.gasSpendUSD}"

'Gas Spend USD: $6.434590946335803'

In [16]:
f"New Shares Amount: {position.shares()[1]}"

'New Shares Amount: 8.599732233561554'

### Withdraw 2 Shares from the Position (Using the 'Convert All' strategy):

* Since we are using the 'Convert All' strategy, we need to specify the percentage of tokens returned that should be the stable token

   In this case, we will use 1.0 (100%) to return all owed tokens in the stable token.

In [12]:
# Must supply the shares in integer value:
shares = position.to_wei(2, position.bep20_vault_token.decimals())
shares

2000000000000000000

In [17]:
# We don't need to specify a strategy here, since the default strategy is "Minimize Trading":
txn_receipt = position.do_withdraw(shares, pct_stable=1.0, strategy="convert all")
txn_receipt

{'value': 0, 'gas': 5334790, 'from': '0xC9E6e248928aC18eD6b103653cBcF66d23B743C6', 'chainId': 56, 'gasPrice': 5000000000, 'nonce': 18, 'to': '0x0256E784f73391797f80a9902c0fD05a718a812a', 'data': '0xe116d2020000000000000000000000000000000000000000000000001bc16d674ec80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000e00000000000000000000000000000000000000000000000000000000000002710000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000190000000000000000000000000000000000000000000000000000000000000001'}


TransactionReceipt(transactionHash=HexBytes('0x568a271e078cb47f8310ad67d7d4924d12aa2f25541267bec00ccb45f0bda08a'), blockHash=HexBytes('0xfaf6c7eb0ec068ab0ea16787a66a2ecdbe3c609d33de18f5290e48392dfbd132'), blockNumber=19784754, contractAddress=None, cumulativeGasUsed=8573473, gasSpendUSD=6.960145252238839, fromAddress='0xC9E6e248928aC18eD6b103653cBcF66d23B743C6', toAddress='0x0256E784f73391797f80a9902c0fD05a718a812a', status=1, transactionIndex=39, type='0x0', effectiveGasPrice=None)

In [18]:
f"Gas Spend USD: ${txn_receipt.gasSpendUSD}"

'Gas Spend USD: $6.960145252238839'

In [19]:
f"New Shares Amount: {position.shares()[1]}"

'New Shares Amount: 6.599732233561553'